In [1]:
import fastai; fastai.__version__

'1.0.46.dev0'

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.vision import *
import string

# create data

In [3]:
path = untar_data(URLs.PETS, dest='/data/fastai/')

path.ls()

[PosixPath('/data/fastai/oxford-iiit-pet/other'),
 PosixPath('/data/fastai/oxford-iiit-pet/annotations'),
 PosixPath('/data/fastai/oxford-iiit-pet/models'),
 PosixPath('/data/fastai/oxford-iiit-pet/images')]

In [4]:
path_anno = path/'annotations'
path_img = path/'images'

In [5]:
# fastai
pat = r'/([^/]+)_\d+.jpg$'
pat = re.compile(pat)
def _get_label(fn): 
    if isinstance(fn, Path): fn = fn.as_posix()
    res = pat.search(str(fn))
    assert res,f'Failed to find "{pat}" in "{fn}"'
    return res.group(1)

In [6]:
imgs = np.random.choice([o for o in path_img.ls() if o.suffix=='.jpg'], 100)

np.random.seed(42)
cat1 = [_get_label(o) for o in imgs]
cat2 = np.random.choice(list(['A', 'B']), len(imgs))
mcat1 = [np.random.choice(list(['x', 'y', 'z']), np.random.choice(range(3)), replace=False) for _ in range(len(imgs))]
mcat2 = [np.random.choice(list(['ccc', 'sss', 'nnn', 'www']), np.random.choice(range(4)), replace=False) 
         for _ in range(len(imgs))]
num1 = [np.random.randn() for _ in range(len(imgs))]
num2 = [np.random.rand() for _ in range(len(imgs))]

df_full = pd.DataFrame({'img': [o.stem for o in imgs], 
                  'cat1': cat1, 'cat2': cat2, 
                   'mcat1': [" ".join(o) for o in mcat1],
                   'mcat2': [" ".join(o) for o in mcat2],
                   'num1': num1,
                   'num2': num2,
                  })
df_full['is_valid'] = np.random.choice([True,False], len(df_full), p=[0.2, 0.8])

In [7]:
df_full.shape

(100, 8)

In [8]:
df_full.sample(10)

,img,cat1,cat2,mcat1,mcat2,num1,num2,is_valid
27,german_shorthaired_176,german_shorthaired,B,y,www ccc nnn,-1.040850,0.511657,False
44,Sphynx_9,Sphynx,B,,,-0.107985,0.687166,True
3,Sphynx_14,Sphynx,A,z x,,0.733173,0.400300,False
60,english_cocker_spaniel_5,english_cocker_spaniel,A,x,www nnn,-0.656529,0.473970,False
15,Russian_Blue_66,Russian_Blue,A,z x,www,-0.103841,0.347619,False
83,beagle_115,beagle,B,x,ccc,0.116411,0.805865,False
63,Persian_200,Persian,A,x z,nnn ccc,-0.671024,0.479582,False
49,yorkshire_terrier_125,yorkshire_terrier,B,y,nnn ccc,1.232514,0.354147,False
93,staffordshire_bull_terrier_66,staffordshire_bull_terrier,B,x y,sss,-0.144049,0.005300,False
18,Egyptian_Mau_45,Egyptian_Mau,B,x y,ccc sss,-0.701813,0.534424,False


In [9]:
tfms = get_transforms()

# single task (using existing functionalities)

## single-class

In [325]:
df=df_full[['img', 'cat1', 'is_valid']]
df.head()

,img,cat1,is_valid
0,scottish_terrier_17,scottish_terrier,False
1,keeshond_169,keeshond,False
2,great_pyrenees_190,great_pyrenees,False
3,Ragdoll_56,Ragdoll,False
4,Ragdoll_152,Ragdoll,False


### databunch factory method

In [280]:
data = ImageDataBunch.from_df(path, df, folder='images', suffix='.jpg', size=64)

/data/anaconda3/envs/fa1043/lib/python3.7/site-packages/fastai/data_block.py:486: UserWarning: You are labelling your items with CategoryList.
Your valid set contained the following unknown labels, the corresponding items have been discarded.
yorkshire_terrier, Ragdoll
  if getattr(ds, 'warn', False): warn(ds.warn)


In [281]:
learn = create_cnn(data, models.resnet18, metrics=error_rate)
learn.loss_func

FlattenedLoss of CrossEntropyLoss()

In [282]:
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,error_rate
1,4.018491,3.704471,0.833333


### data block

In [32]:
data = (ImageItemList.from_df(df, path, folder='images', suffix='.jpg')
        .split_from_df(col='is_valid')
        .label_from_df(cols='cat1')
        .transform(tfms, size=64)).databunch()

> /data/git/fastai/fastai/data_block.py(238)_label_from_list()
-> y = label_cls(labels, path=self.path, **kwargs)
(Pdb) q


BdbQuit: 

In [257]:
learn = create_cnn(data, models.resnet18, metrics=error_rate)
learn.loss_func

FlattenedLoss of CrossEntropyLoss()

In [258]:
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,error_rate
1,4.233921,4.636864,1.000000


## multiclass 

In [33]:
df=df_full[['img', 'mcat1', 'is_valid']]
df.head()

,img,mcat1,is_valid
0,american_bulldog_115,x z,False
1,scottish_terrier_170,x,False
2,chihuahua_21,x,False
3,Sphynx_14,z x,False
4,shiba_inu_195,,False


### databunch factory method

In [273]:
data = ImageDataBunch.from_df(path, df, folder='images', suffix='.jpg', size=64, label_delim=' ')

In [274]:
learn = create_cnn(data, models.resnet18)
learn.loss_func

FlattenedLoss of BCEWithLogitsLoss()

In [275]:
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss
1,0.868255,0.864794


### data block

In [276]:
data = (ImageItemList.from_df(df, path, folder='images', suffix='.jpg')
        .split_from_df(col='is_valid')
        .label_from_df(cols='mcat1', label_delim=' ')
        .transform(tfms, size=64)).databunch()

In [34]:
# debug
src=(ImageItemList.from_df(df, path, folder='images', suffix='.jpg')
        .split_from_df(col='is_valid'))
def foo():
    breakpoint()
    src.label_from_df(cols='mcat1', label_delim=' ')
foo()

> <ipython-input-34-5c6382146a5e>(6)foo()
-> src.label_from_df(cols='mcat1', label_delim=' ')
(Pdb) 
(Pdb) 
(Pdb) 
(Pdb) n
--Return--
> <ipython-input-34-5c6382146a5e>(6)foo()->None
-> src.label_from_df(cols='mcat1', label_delim=' ')
(Pdb) c


In [277]:
learn = create_cnn(data, models.resnet18)
learn.loss_func

FlattenedLoss of BCEWithLogitsLoss()

In [278]:
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss
1,0.840250,0.740074


## regression

In [320]:
df=df_full[['img', 'num1', 'is_valid']]
df.head()

,img,num1,is_valid
0,scottish_terrier_17,0.851944,False
1,keeshond_169,0.768339,False
2,great_pyrenees_190,0.554693,False
3,Ragdoll_56,0.733173,False
4,Ragdoll_152,-1.104222,False


### databunch factory method

In [321]:
# doesn't work
# data = ImageDataBunch.from_df(path, df[['img', 'num1']], folder='images', suffix='.jpg', size=64)

### data block

In [322]:
data = (ImageItemList.from_df(df, path, folder='images', suffix='.jpg')
        .split_from_df(col='is_valid')
        .label_from_df(cols='num1')
        .transform(tfms, size=64)).databunch()

In [323]:
learn = create_cnn(data, models.resnet18, metrics=mse)
learn.loss_func

FlattenedLoss of MSELoss()

In [324]:
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,mean_squared_error
1,2.238382,1.670990,1.670990


# multi task

https://docs.fast.ai/tutorial.itemlist.html

## 2 single-classes

In [10]:
df=df_full[['img', 'cat1', 'cat2', 'is_valid']]
df.head()

,img,cat1,cat2,is_valid
0,american_bulldog_115,american_bulldog,A,False
1,scottish_terrier_170,scottish_terrier,B,False
2,chihuahua_21,chihuahua,A,False
3,Sphynx_14,Sphynx,A,False
4,shiba_inu_195,shiba_inu,A,False


data_block.py
```
class CategoryProcessor(PreProcessor):
    "`PreProcessor` that create `classes` from `ds.items` and handle the mapping."

class CategoryListBase(ItemList):
    "Basic `ItemList` for classification."

class CategoryList(CategoryListBase):
    "Basic `ItemList` for single classification labels."

class ItemList():
    "A collection of items with `__len__` and `__getitem__` with `ndarray` indexing semantics."
    _bunch,_processor,_label_cls,_square_show,_square_show_res = DataBunch,None,None,False,False
    def get_label_cls(self, labels, label_cls:Callable=None, label_delim:str=None, **kwargs):
        "Return `label_cls` or guess one from the first element of `labels`."
        if label_cls is not None:               return label_cls

    def _label_from_list(self, labels:Iterator, label_cls:Callable=None, from_item_lists:bool=False, **kwargs)->'LabelList':
        "Label `self.items` with `labels`."
        label_cls = self.get_label_cls(labels, label_cls=label_cls, **kwargs)

    def label_from_df(self, cols:IntsOrStrs=1, label_cls:Callable=None, **kwargs):
        "Label `self.items` from the values in `cols` in `self.inner_df`."
        return self._label_from_list(_maybe_squeeze(labels), label_cls=label_cls, **kwargs)
    
    def _label_from_list(self, labels:Iterator, label_cls:Callable=None, from_item_lists:bool=False, **kwargs)->'LabelList':
        "Label `self.items` with `labels`."
        label_cls = self.get_label_cls(labels, label_cls=label_cls, **kwargs)
        y = label_cls(labels, path=self.path, **kwargs)
```

core.py
```
class Category(ItemBase):
    "Basic class for single classification labels."
```

https://docs.fast.ai/tutorial.itemlist.html#Example:-ImageTuple
```
class ImageTuple(ItemBase):
    def __init__(self, img1, img2):
        self.img1,self.img2 = img1,img2
        self.obj,self.data = (img1,img2),[-1+2*img1.data,-1+2*img2.data]
```

https://docs.fast.ai/tutorial.itemlist.html#Example:-ImageTuple
```
class ImageTuple(ItemBase):
    def __init__(self, img1, img2):
        self.img1,self.img2 = img1,img2
        self.obj,self.data = (img1,img2),[-1+2*img1.data,-1+2*img2.data]
```

In [25]:
class MultitaskTarget(ItemBase):
    "Basic class for multitask target."
    def __init__(self,data,obj): self.data,self.obj = data,obj
    def __int__(self): return int(self.data)
    def __str__(self): return str(self.obj)

class MultitaskProcessor(CategoryProcessor):
    

In [26]:
class MultitaskTargetList(ItemBase):
    "Basic class for multitask target."
    def __init__(self,data,obj): self.data,self.obj = data,obj
    def __int__(self): return int(self.data)
    def __str__(self): return str(self.obj)


In [12]:
src = (ImageItemList.from_df(df, path, folder='images', suffix='.jpg')
        .split_from_df(col='is_valid'))

In [23]:
src.label_from_df(cols=['cat1', 'cat2'], label_cls=MultitaskTarget)

> /data/git/fastai/fastai/data_block.py(238)_label_from_list()
-> y = label_cls(labels, path=self.path, **kwargs)
(Pdb) label_cls
<class '__main__.MultitaskTarget'>
(Pdb) q


BdbQuit: 

In [14]:
def foo():
    breakpoint()
    src.label_from_df(cols=['cat1', 'cat2'], label_cls=MultitaskTarget)

In [22]:
foo()

> <ipython-input-14-d119a3027a84>(3)foo()
-> src.label_from_df(cols=['cat1', 'cat2'], label_cls=MultitaskTarget)
(Pdb) c
> /data/git/fastai/fastai/data_block.py(238)_label_from_list()
-> y = label_cls(labels, path=self.path, **kwargs)
(Pdb) q


BdbQuit: 

In [161]:
class MultitaskTarget(ItemBase):
    "Basic class for multitask target."
    def __init__(self,target): 
        self.target = target
        self.data = []
        
    def __str__(self): 
        return str(self.target)

def multitask_loss():
    return 0

class MultitaskTargetList():
    def __init__(self, items:Iterator, 
                 classes1:Collection=None, 
                 classes1:Collection=None, 
                 mclasses1:Collection=None, 
                 mclasses1:Collection=None, 
                 label_delim:str=None, **kwargs):
        super().__init__(items, classes=classes, **kwargs)
        self.loss_func = multitask_loss

    def get(self, i):
        o = self.items[i]
        if o is None: return None
        return MultitaskTarget(o, self.classes[o])
    
    

In [162]:
target = df.to_dict('records')[np.random.choice(len(df))]
del target['img']
target

{'cat1': 'chihuahua',
 'cat2': 'O',
 'mcat1': 's e z',
 'mcat2': 'abc xyz',
 'num1': -0.10838220224884176,
 'num2': 0.38568197098798007}

In [163]:
MultitaskTarget(target)

MultitaskTarget {'cat1': 'chihuahua', 'cat2': 'O', 'mcat1': 's e z', 'mcat2': 'abc xyz', 'num1': -0.10838220224884176, 'num2': 0.38568197098798007}

```
class CategoryList(CategoryListBase):
    "Basic `ItemList` for single classification labels."
    _processor=CategoryProcessor
    def __init__(self, items:Iterator, classes:Collection=None, label_delim:str=None, **kwargs):
        super().__init__(items, classes=classes, **kwargs)
        self.loss_func = CrossEntropyFlat()

    def get(self, i):
        o = self.items[i]
        if o is None: return None
        return Category(o, self.classes[o])

    def analyze_pred(self, pred, thresh:float=0.5): return pred.argmax()

    def reconstruct(self, t):
        return Category(t, self.classes[t])
```